# Exploring [Commercial Vibrancy in Chicago](https://www.jpmorganchase.com/corporate/institute/insight-commercial-vibrancy-of-chicago.htm)

In March of 2017, the [JPMorgan Chase Institute](https://www.jpmorganchase.com/corporate/institute/institute.htm) released a report called [**Going the Distance:** Big Data on Resident Access to Everyday Goods](https://www.jpmorganchase.com/corporate/institute/document/institute-access-to-consumption-brief.pdf). The report leveraged the ability to capture the distance between a consumer's home zip code, and the zip code in which the merchant resided, for millions of transactions. We refer to this distance as the "merchant distance". When merchant distances are collected within a city, we can analyze how they differ in different parts of the city, and for different populations. Such analysis yields a novel view of within-city demand; it also has the power to speak to inequality in access to amenities across subpopulations, offering a new window into the lived experience of millions of people.

The initial report focused on Detroit and New York, but a more recent analysis has centered on Chicago. The analysis performed yielded a [publicly available data file](https://www.jpmorganchase.com/corporate/institute/document/chicago_data_2016q2.zip) which holds the median "merchant distance" by zip code across all observed purchases. It also holds median merchant distances for grocery and restaurant purchases, in addition to overall merchant distances for low- and high-income residents of Chicago.

In this Notebook, we will create a map of this data, to better the commercial landscape in Chicago.

In [31]:
# Data manipulation
import numpy as np
import pandas as pd
import geopandas as gpd
import json
# Visualization
import bokeh.plotting as bp
import bokeh.models as bm
import bokeh.io as bio
from bokeh.tile_providers import CARTODBPOSITRON_RETINA
import IPython.display as ipd
# Color mapping
import matplotlib as mpl
import matplotlib.cm as cmap

# Define function to allow us to display charts in an iframe
def show_iframe(url, iheight=400, iwidth=1000):
    display_string = '<iframe src={url} width={w} height={h}></iframe>'.format(url=url, w=iwidth, h=iheight)
    print(display_string)
    return ipd.HTML(display_string)

## Data Read

The data we need for this exercise comes from two sources. The first is just the "flat" CSV file (referenced above) that contains the access data by zip code in Chicago. This would be all we needed if we were concerned only with making non-spatial visualizations. However, since we are going to make a map, we need some way of associating the zip code level data with actual shapes that represent zip codes in Chicago. To be more precise, we need shapes represented as a set of coordinates, and then we need to associated specific data values with each coordinate set. We have taken the liberty of already downloading the flat file and depositing it in `../data/chicago_data_2016q2.csv`. Note that missing values indicate that we did not see enough customers and transactions in a given zip code to clear our minimum reporting threshold.

In [14]:
# Read in data
access = pd.read_csv('../data/chicago_data_2016q2.csv')

access.head()

,zip_code,overall,grocery_stores,restaurants,low_income,high_income,cash_buffer_days
0,60601,0.5,NaN,0.6,0.5,0.5,17
1,60602,0.8,1.0,0.7,NaN,1.0,19
2,60603,0.9,1.2,0.6,NaN,0.9,17
3,60604,0.9,NaN,0.8,NaN,NaN,20
4,60605,1.0,NaN,1.1,1.0,1.0,16


To get the spatial information, we are going to use a [GeoJSON](http://geojson.org/) file (`.geojson`) because they are easier to work with in Python.  (Check out this [great post](https://macwright.org/2015/03/23/geojson-second-bite) about GeoJSON in depth.) Remember that [JSON](http://www.json.org/) files (of which GeoJSON is a special case) are structured in a very similar way to [Python dictionaries](https://docs.python.org/3/tutorial/datastructures.html#dictionaries). Consequently, once we read them in, we have all kinds of great built-in methods for manipulating them. The tool set for working with shapefiles (`.shp`) files is just not as robust. That said, shapefiles are far more ubiquitous than GeoJSON files because [ESRI](https://www.esri.com/en-us/home) has spent years saturating the market with their (initially) proprietary format. This is likely the reason that all of the data that the Census offers in its [Topologically Integrated Geographic Encoding and Referencing](https://www.census.gov/geo/maps-data/data/tiger.html) (TIGER) products are available as shapefiles.  Consequently, we will need to convert our spatial data files from shapefiles to GeoJSON. 

To prepare this example, I have used the [Geospatial Data Abstraction Library](http://gdal.org/) (GDAL) to convert the Census ZCTA5 shapefile to GeoJSON:

    ogr2ogr -f GeoJSON -t_srs crs:84 tl_2017_us_zcta510.geojson tl_2017_us_zcta510.shp
    
That said, you don't have to install GDAL to convert the file you download from the Census. If you like, you can use one of the many available online converters that can accept shapefiles and return GeoJSON. (Literally just search "convert shapefile to geojson", and Google will know where you are going halfway through "shapefile".)  Let's take a quick look at this puppy.

In [15]:
!ls -la ../data/spatial/
!head -5 ../data/spatial/tl_2017_us_zcta510.geojson

total 2757824
drwxr-xr-x 2 root     root           4096 Mar 25 17:58 .
drwxr-xr-x 3 root     root           4096 Mar 25 16:09 ..
-rw-r--r-- 1 root     root         770061 Mar 25 20:45 chicago_zips_2017.geojson
-rwxrwxr-x 1 root     root              5 Sep 18  2017 tl_2017_us_zcta510.cpg
-rwxrwxr-x 1 root     root        2320402 Sep 18  2017 tl_2017_us_zcta510.dbf
-rw-r--r-- 1 root     root     1439602034 Mar 25 16:18 tl_2017_us_zcta510.geojson
-rwxrwxr-x 1 root     root            165 Sep 18  2017 tl_2017_us_zcta510.prj
-rwxrwxr-x 1 root     root      848204320 Sep 18  2017 tl_2017_us_zcta510.shp
-rwxrwxrwx 1 root     root          15858 Sep 18  2017 tl_2017_us_zcta510.shp.ea.iso.xml
-rwxrwxrwx 1 root     root          38581 Sep 18  2017 tl_2017_us_zcta510.shp.iso.xml
-rwxrwxrwx 1 root     root          18581 Sep 18  2017 tl_2017_us_zcta510.shp.xml
-rwxrwxr-x 1 root     root         265252 Sep 18  2017 tl_2017_us_zcta510.shx
-rw-rw-r-- 1 choct155 choct155  532724352 Mar 25 15:55 tl_201

In [16]:
# !tail -3 ../data/spatial/tl_2017_us_zcta510.geojson

The first thing to note is the size of the GeoJSON file.  It is a rather beastly 1.4 GB, so we definitely do not want to read this whole thing into memory.  (Here are some [handy tips](https://www.dataquest.io/blog/python-json-tutorial/) about working with large JSON assets.) We need to find some way of reading in only Chicago zip codes. The second call (which prints out the head of the GeoJSON file) gives us clues as to how to achieve this goal.  The general structure of the file is as follows:

    {
        "type": "FeatureCollection",
        "crs": { 
            "type": "name", 
            "properties": { ... } 
            },
        "features": [
            { 
                "type": "Feature", 
                "properties": { ... }, 
                "geometry": { 
                    "type": "Polygon", 
                    "coordinates": [ ... ]
                 }
            }
        ]
    }
    
The vast majority of the information in this structure resides in `f["features"]["geometry"]["coordinates"]`. To make sure we only carry the coordinates we need, we should leverage the filtering opportunity that `f["features"]["properties"]` provides. If we take a closer look at the first feature, we can see all the data values offered for each zip code:

    "properties": { 
        "ZCTA5CE10": "84049", 
        "GEOID10": "84049", 
        "CLASSFP10": "B5", 
        "MTFCC10": "G6350", 
        "FUNCSTAT10": "S", 
        "ALAND10": 215986428, 
        "AWATER10": 10468874, 
        "INTPTLAT10": "+40.5228727", 
        "INTPTLON10": "-111.5243678" 
    }
    
Hopefully `ZCTA5CE0` sticks out as a high-value target, since our flat file already gave us a list of the ZIP codes we can map.

In [17]:
# Grab the set of zip codes
zips = sorted(set(access['zip_code']))

print(zips)

[60601, 60602, 60603, 60604, 60605, 60606, 60607, 60608, 60609, 60610, 60611, 60612, 60613, 60614, 60615, 60616, 60617, 60618, 60619, 60620, 60621, 60622, 60623, 60624, 60625, 60626, 60628, 60629, 60630, 60631, 60632, 60633, 60634, 60636, 60637, 60638, 60639, 60640, 60641, 60642, 60643, 60644, 60645, 60646, 60647, 60649, 60651, 60652, 60653, 60654, 60655, 60656, 60657, 60659, 60660, 60661]


Our goal is to build a list of features that corresponds to only our Chicago zip codes. At a high level, we need to do the following:

1. Create a list of Chicago zip codes
2. Loop over the zip codes, scanning the file for lines that contain each one
3. If a line holds a Chicago zip code, deposit the feature in a list and move to the next zip code. If it doesn't, simply move to the next one.

Yet again, behold the power of IPython's ability to blend bash commands with Python logic and syntax.  We don't want to read in the whole file before filtering, because that will take forever. Furthermore, bash already has an excellent tool for finding substrings of relevance ("IPython" is a substring of "IPython for the win"): [grep](https://www.gnu.org/savannah-checkouts/gnu/grep/manual/grep.html).  To provide an example of the core functionality that will make this work, note how easy and fast it is to find the feature information pertaining to the first Chicago zip code, `60601`.

In [18]:
first_zip = !grep '"ZCTA5CE10": "60601"' ../data/spatial/tl_2017_us_zcta510.geojson

first_zip

['{ "type": "Feature", "properties": { "ZCTA5CE10": "60601", "GEOID10": "60601", "CLASSFP10": "B5", "MTFCC10": "G6350", "FUNCSTAT10": "S", "ALAND10": 934226, "AWATER10": 60682, "INTPTLAT10": "+41.8856419", "INTPTLON10": "-087.6215226" }, "geometry": { "type": "Polygon", "coordinates": [ [ [ -87.633965, 41.886985 ], [ -87.633727, 41.886979 ], [ -87.633389, 41.886982 ], [ -87.632726, 41.886972 ], [ -87.632486, 41.886974 ], [ -87.632486, 41.886993 ], [ -87.632487, 41.887062 ], [ -87.631008, 41.887063 ], [ -87.63101, 41.887098 ], [ -87.631015, 41.88727 ], [ -87.631019, 41.887383 ], [ -87.631022, 41.887473 ], [ -87.631023, 41.88751 ], [ -87.630725, 41.887509 ], [ -87.629834, 41.887508 ], [ -87.629537, 41.887508 ], [ -87.62923, 41.887507 ], [ -87.62831, 41.887506 ], [ -87.628004, 41.887506 ], [ -87.62794, 41.887506 ], [ -87.627896, 41.88752 ], [ -87.627584, 41.887626 ], [ -87.62748, 41.887662 ], [ -87.627365, 41.8877 ], [ -87.62707, 41.887912 ], [ -87.626972, 41.887983 ], [ -87.626882, 41.88

We are now going to use this compact approach to populate features in a JSON string modeled after the original GeoJSON file. Then we can add the GeoJSON front end to our feature list, make sure we cap off the features with closing brackets, and we will be in business.

In [19]:
# Create dictionary to hold new subset of spatial info
chicago = '''
{
    "type": "FeatureCollection",
    "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
    "features": [
'''

# Create container to hold feature-specific substrings
chi_zips = []
# For each Chicago zip code
for z in zips:
    print('Scanning for {}'.format(z))
    # grab the relevant feature info and throw it in the chicago dict
    z_tmp = !grep '"ZCTA5CE10": "{z}"' ../data/spatial/tl_2017_us_zcta510.geojson
    chi_zips.append(z_tmp[0][:-1]) # bash results are collected as a list, and we want to drop trailing commas
    
chicago_json = chicago + (',').join(chi_zips) + ']}'

Scanning for 60601
Scanning for 60602
Scanning for 60603
Scanning for 60604
Scanning for 60605
Scanning for 60606
Scanning for 60607
Scanning for 60608
Scanning for 60609
Scanning for 60610
Scanning for 60611
Scanning for 60612
Scanning for 60613
Scanning for 60614
Scanning for 60615
Scanning for 60616
Scanning for 60617
Scanning for 60618
Scanning for 60619
Scanning for 60620
Scanning for 60621
Scanning for 60622
Scanning for 60623
Scanning for 60624
Scanning for 60625
Scanning for 60626
Scanning for 60628
Scanning for 60629
Scanning for 60630
Scanning for 60631
Scanning for 60632
Scanning for 60633
Scanning for 60634
Scanning for 60636
Scanning for 60637
Scanning for 60638
Scanning for 60639
Scanning for 60640
Scanning for 60641
Scanning for 60642
Scanning for 60643
Scanning for 60644
Scanning for 60645
Scanning for 60646
Scanning for 60647
Scanning for 60649
Scanning for 60651
Scanning for 60652
Scanning for 60653
Scanning for 60654
Scanning for 60655
Scanning for 60656
Scanning for

In [20]:
# print(chicago_json)

We can hold these data for future use by writing the subset back to disk.

In [21]:
with open('../data/spatial/chicago_zips_2017.geojson', 'w') as outfile:
    outfile.write(chicago_json)

## Data Inspection

Now that we have our GeoJSON string for Chicago, let's visually inspect our boundaries to see if they make sense.  We can use a quick and dirty polygon plot.

In [22]:
# Define GeoJSON source
chi_src = bm.GeoJSONDataSource(geojson=chicago_json)

# Define location of output file
bp.output_file('../figs/chicago_access.html')

# Plot blue polys with white lines
p = bp.figure(height=900, width=900, lod_threshold=1)
p.patches(xs='xs', ys='ys', fill_color='blue', source=chi_src)
p.multi_line(xs='xs', ys='ys', line_color='white', line_width=0.3, source=chi_src)

# Show the figure
bp.save(p)

show_iframe('../figs/chicago_access.html', iwidth=950, iheight=850)

<iframe src=../figs/chicago_access.html width=950 height=850></iframe>


## Joining Access Data to Spatial Data

One of the most useful projects I have come across for spatial analysis has been [GeoPandas](http://geopandas.org/). It's kind of exactly what it sounds like. This project combines spatial data operations with the flexible pandas DataFrame. Among other things, it makes it much more convenient to join flat data to polygons, so we will use it to assign access values (and associated colors) to each zip code. 

First, let's create a consolidated GeoDataFrame.

In [23]:
import matplotlib as mpl

In [84]:
# Read in chicago zips into a GeoDataFrame
chi_df = gpd.read_file('../data/spatial/chicago_zips_2017.geojson').rename(columns = {'ZCTA5CE10': 'zip_code'})

# Set the projection to be consistent with Bokeh tiles (Mercator)
chi_df = chi_df.to_crs({'init': 'epsg:3857'})

# Set zip code to index to facilitate joining
for df in [access, chi_df]:
    if 'zip_code' in df.columns:
        # Make sure zip codes are integers in both sets
        df['zip_code'] = df['zip_code'].astype(int)
        df.set_index('zip_code', inplace=True)
        
# Join the access data to the spatial data
for c in access.columns:
    chi_df[c] = access[c].loc[chi_df.index]


Second, we need to choose colors for the different values we observe. We want color intensity to have consistent meaning across each group.  In other words, long distances for grocery store purchases should map (at least in intensity) to long distances in restaurants. Note that this is an arguable point, because it imposes restrictions on us in a sense. If we map intensity across different variables, we should probably use the same color to reinforce the point. In this particular instance, we will partially violate this idea by using one color to map product types (e.g grocery stores and restaurants), and another to map income groups (e.g. low- and high-income).

In [85]:
# Capture minimum and maximum values within each driver bucket
overall_min = access['overall'].min()
overall_max = access['overall'].max()
prod_type_min = access[['grocery_stores', 'restaurants']].min().min()
prod_type_max = access[['grocery_stores', 'restaurants']].max().max()
inc_grp_min = access[['low_income', 'high_income']].min().min()
inc_grp_max = access[['low_income', 'high_income']].max().max()

# Capture normalizer objects (they map data to the [0,1] interval)
norm = {
    'overall': mpl.colors.Normalize(vmin=overall_min, vmax=overall_max),
    'prod_type': mpl.colors.Normalize(vmin=prod_type_min, vmax=prod_type_max),
    'inc_grp': mpl.colors.Normalize(vmin=inc_grp_min, vmax=inc_grp_max)
}

# Generate color mappers
cmapper = {
    'overall': cmap.ScalarMappable(norm=norm['overall'], cmap='PiYG'),
    'prod_type': cmap.ScalarMappable(norm=norm['prod_type'], cmap='RdBu'),
    'inc_grp': cmap.ScalarMappable(norm=norm['inc_grp'], cmap='PuOr')
}

# Map colors to values
chi_df['overall_color'] = chi_df['overall'].apply(lambda x: mpl.colors.to_hex(cmapper['overall'].to_rgba(x)))
for c in ['grocery_stores', 'restaurants']:
    chi_df[c + '_color'] = chi_df[c].apply(lambda x: mpl.colors.to_hex(cmapper['prod_type'].to_rgba(x)))
for c in ['low_income', 'high_income']:
    chi_df[c + '_color'] = chi_df[c].apply(lambda x: mpl.colors.to_hex(cmapper['inc_grp'].to_rgba(x)))

## A Basic Choropleth

Now we are in a position to get some preliminary insight.  For the time being, we will stick with evaluating overall merchant distances, until we get the functionality we need.  Since we have several variables, we will seek to build a function that takes a variable as an argument.

In [101]:
# Define data source
chi_access_src = bm.GeoJSONDataSource(geojson=chi_df.reset_index().to_json())

def plot_dist(v, height_in=900, width_in=750):
    # Define location of output file
    outf = '../figs/chicago_access_{}.html'.format(v)
    bp.output_file(outf)
    # Define figure
    fig = bp.figure(title='Median Merchant Distance: {}'.format(v.title()),
                       height=height_in, width=width_in)
    # Add zip polygons
    fig.patches(xs='xs', ys='ys', alpha=0.9, source=chi_access_src,
                fill_color={'field': v+'_color'}, line_width=1, line_color='#626770')
    # Define hover appearance (in raw HTML)
    hover_html = '''
    <div><b>ZIP Code</b>: @zip_code</div>
    <div><b>Merchant Distance</b>: @{}</div>
    '''.format(v)
    hover = bm.HoverTool(
        point_policy='follow_mouse',
        tooltips=(hover_html)
    )
    # Define tap tool
    tap = bm.TapTool()
    fig.add_tools(hover, tap)
    # Add map tile
    fig.add_tile(CARTODBPOSITRON_RETINA)
    # Get rid of axis lines
    fig.xaxis.visible = False
    fig.yaxis.visible = False
    fig.grid.visible = False
    # Show the figure
    bp.save(fig)
    return outf

ovr = plot_dist('overall')
show_iframe(ovr, iwidth=850, iheight=950)

<iframe src=../figs/chicago_access_overall.html width=850 height=950></iframe>


Now we are cooking with gas.

In [102]:
import geojsonio as gjio

In [103]:
gjio.display(chicago_json)

UnprocessableEntity: 422 Validation Failed